<a href="https://colab.research.google.com/github/vsolodkyi/NeuralNetworks_SkillBox/blob/main/module_13/mid_ml_nlp_les_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Урок 3 Векторизация текста: Bag of Words

Итак, мы умеем подготавливать текст к обработке: приводить слова к начальным формам, разделять текст на токены, удалять "мусорные" токены (стоп-слова). Однако, мы знаем, что нейросети работают не с текстом, а с числами. Давайте разбираться, как переводить токены в числа, то есть с тем, как работает векторизация.

Bag of Words - это способ перейти от набора токенов к численному вектору. Алгоритм векторизации текста по модели BoW:

1. определяем количество $N$ различных токенов во всех доступных текста - так называемый "словарь".
1. присваиваем каждому токену случайный номер от $0$ до $N$.
1. для каждого документа $i$ формируем вектор размерности $N$ - ставим на позицию $j$ количество вхождений токена с номером $j$, которые содержатся в тексте $i$.

Каждый токен мы по сути представляем в виде вектора размерности $N$, который состоит из нулей и всего одной единицы, такое кодирование называется *One-Hot encoding*. А каждый документ это "сумма" всех one-hot векторов входящих в него токенов.

Такой подход хорошо иллюстрируется картинкой:

![bow](https://sun9-2.userapi.com/c854228/v854228722/1f4c57/BWDIDvXh-ew.jpg)

Для каждого элемента получаем вектор из нулей и единиц. При этом размерность словаря обычно составляет несколько десятков тысяч, а количество токенов в одном документе несколько десятков - то есть нулей значительно больше, чем единиц - такие данные называются *разреженными*.

В таком виде данные уже пригодны для работы с нейросетью или любым другим алгоритмом ML, однако есть несколько довольно простых и полезных вещей, которые мы можем сделать и без нейросетей. Давайте сначала разберем их, а потом вернемся к нейросетям. Такое представление текста позволяет решать интересные задачи - например, находить самые похожие друг на друга тексты. Чтобы как-то формализовать понятие "схожести" текстов, вводится понятие *косинусного расстояния* между двумя векторами текстов $a$ и $b$ размерности $N$. С этой метрикой вы [можете познакомиться в Википедии](https://ru.wikipedia.org/wiki/Векторная_модель#Косинусное_сходство ), формула такая для двух векторов $a$ и $b$ с координатами $a_i$ и $b_i$ соответственно:
$$
\text{similarity} = \cos (\theta) = 1 - \frac{\sum_{i=1}^{N}a_ib_i}{\sqrt{\sum_{i=1}^{N}(a_i)^2}\sqrt{\sum_{i=1}^{N}(b_i)^2}}
$$

Интуитивное объяснение для простого случая: два документа полностью совпадают, тогда единички в них стоят на одних и тех же местах - расстояние между ними будет нулевым. Если два текста совершенно не пересекаются, то единички будут стоять на разных местах - расстояние в этом случае равно единице. Самостоятельно реализовывать функцию не нужно - есть готовая реализация в [scipy.spatial.distance.cosine](https://docs.scipy.org/doc/scipy-0.14.0/reference/generated/scipy.spatial.distance.cosine.html)

Векторизуем наш корпус (набор текстов) с помощью класса `CountVectorizer()` (то есть превращаем наборы токенов в наборы векторов)

In [16]:
import nltk
import string

# дополнительный словарь со знаками пунктуации
nltk.download('punkt', download_dir='.')

[nltk_data] Downloading package punkt to ....
[nltk_data]   Package punkt is already up-to-date!


True

In [5]:
import pandas as pd

df = pd.read_csv('data/brand_tweets.csv', sep=',', encoding='utf8')
# удаляем строки, в которых отсутствует текст твита
df.drop(df[df.tweet_text.isnull()].index, inplace=True)
print(df.shape)

df.head()

(3904, 3)


,tweet_text,emotion_in_tweet_is_directed_at,is_there_an_emotion_directed_at_a_brand_or_product
0,.@wesley83 I have a 3G iPhone. After 3 hrs twe...,iPhone,Negative emotion
1,@jessedee Know about @fludapp ? Awesome iPad/i...,iPad or iPhone App,Positive emotion
2,@swonderlin Can not wait for #iPad 2 also. The...,iPad,Positive emotion
3,@sxsw I hope this year's festival isn't as cra...,iPad or iPhone App,Negative emotion
4,@sxtxstate great stuff on Fri #SXSW: Marissa M...,Google,Positive emotion


In [6]:
sample_str = df.tweet_text.values[0]

print('== Исходный текст== \n%s\n\n' % sample_str)

tokenized_str = nltk.word_tokenize(sample_str)
print('== Токенизированный текст==\n%s' % tokenized_str)

== Исходный текст== 
.@wesley83 I have a 3G iPhone. After 3 hrs tweeting at #RISE_Austin, it was dead!  I need to upgrade. Plugin stations at #SXSW.


== Токенизированный текст==
['.', '@', 'wesley83', 'I', 'have', 'a', '3G', 'iPhone', '.', 'After', '3', 'hrs', 'tweeting', 'at', '#', 'RISE_Austin', ',', 'it', 'was', 'dead', '!', 'I', 'need', 'to', 'upgrade', '.', 'Plugin', 'stations', 'at', '#', 'SXSW', '.']


In [7]:
tokens = [i.lower() for i in tokenized_str if ( i not in string.punctuation )]
print(tokens)

['wesley83', 'i', 'have', 'a', '3g', 'iphone', 'after', '3', 'hrs', 'tweeting', 'at', 'rise_austin', 'it', 'was', 'dead', 'i', 'need', 'to', 'upgrade', 'plugin', 'stations', 'at', 'sxsw']


In [8]:
stop_words = [
    'i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd",
    'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers',
    'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which',
    'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been',
    'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if',
    'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between',
    'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out',
    'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why',
    'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not',
    'only', 'own', 'same', 'so', 'than', 'too', 'very', 's', 't', 'can', 'will', 'just', 'don', "don't", 'shold',
    "should've", 'now', 'd', 'll', 'm', 'o', 're', 've', 'y', 'ain', 'aren', "aren't", 'couldn', "couldn't",
    'didn', "didn't", 'doesn', "doesn't", 'hadn', "hadn't", 'hasn', "hasn't", 'haven', "haven't", 'isn', "isn't",
    'ma', 'mightn', "mightn't", 'mustn', "mustn't", 'needn', "needn't", 'shan', "shan't", 'shouldn', "shouldn't",
    'wasn', "wasn't", 'weren', "weren't", 'won', "won't", 'wouldn', "wouldn't"
]

filtered_tokens = [i for i in tokens if ( i not in stop_words )]

print(filtered_tokens)

['wesley83', '3g', 'iphone', '3', 'hrs', 'tweeting', 'rise_austin', 'dead', 'need', 'upgrade', 'plugin', 'stations', 'sxsw']


In [9]:
def tokenize_text(raw_text: str):
    """Функция для токенизации текста
    
    :param raw_text: исходная текстовая строка
    """
    tokenized_str = nltk.word_tokenize(raw_text)
    tokens = [i.lower() for i in tokenized_str if ( i not in string.punctuation )]
    filtered_tokens = [i for i in tokens if ( i not in stop_words )]
    return filtered_tokens

# применяем функцию в датафрейму с помощью метода .apply()
tokenized_tweets= df.tweet_text.apply(tokenize_text)

# добавляем новую колонку в исходный датафрейм
df = df.assign(
    tokenized=tokenized_tweets
)

df.tokenized.head()

0    [wesley83, 3g, iphone, 3, hrs, tweeting, rise_...
1    [jessedee, know, fludapp, awesome, ipad/iphone...
2    [swonderlin, wait, ipad, 2, also, should, sale...
3    [sxsw, hope, year, 's, festival, n't, crashy, ...
4    [sxtxstate, great, stuff, fri, sxsw, marissa, ...
Name: tokenized, dtype: object

In [11]:
from sklearn.feature_extraction.text import CountVectorizer

# инициализируем объект, который токенизирует наш текст
# в качестве единственного аргимента передаём функцию, которую мы написали в Уроке 2
# на разбивает каждый документ на токены
vectorizer = CountVectorizer(tokenizer=tokenize_text)
# применяем наш объект-токенизатор к датафрейму с твитами
document_matrix = vectorizer.fit_transform(df.tweet_text.values)
# результат - матрица, в которой находятся числа, строк в мастрице столько, сколько документов
# а столбцов столько, сколько токенов
document_matrix

<3904x7260 sparse matrix of type '<class 'numpy.int64'>'
	with 46060 stored elements in Compressed Sparse Row format>

Класс `sklearn.feature_extraction.text.CountVectorizer` реализует алгоритм преобразования массива текстовых документов в разреженную матрицу такую, что

* число строк совпадает с количеством документов в исходном датафрейме
* количество столбцов совпадает с количеством различных токенов
* объект `CountVectorizer()` содержит в себе разные вспомогательные элементы - например, словарь соответствия токена и его номера

Полученные вектора можно использовать в алгоритмах второго уровня - например, в задаче классификации отзывов.

Пользуясь матрицей, найдем твит, который максимально похож на первый твит из набора:

In [13]:
source_tweet_index = 0
print(df.iloc[source_tweet_index].tweet_text)

.@wesley83 I have a 3G iPhone. After 3 hrs tweeting at #RISE_Austin, it was dead!  I need to upgrade. Plugin stations at #SXSW.


Вычисляем попарные схожести между элементами разреженной матрицы

In [14]:
from sklearn.metrics import pairwise_distances

tweet_distance = 1-pairwise_distances(document_matrix, metric="cosine")

tweet_distance.shape

(3904, 3904)

Мы получили квадратную матрицy, которая содержит столько строк и столбцов, сколько документов в нашем  корпусе  (наборе текстов).

In [15]:
import numpy as np

# отсортируем твиты по “похожести” - чем похожее на source_tweet_index,
# тем ближе к началу списка sorted_similarity
sorted_similarity = np.argsort(-tweet_distance[source_tweet_index,:])

sorted_similarity

array([   0,  633,  420, ..., 2861,  287, 1330])

Мы получили вектор "схожестей", который содержит индексы похожих твитов, расположенных по убыванию схожести. Больше всего твит похож сам на себя, поэтому возьмём индекс второго по схожести элемента (и далее).

In [ ]:
print(df.iloc[0].tweet_text)
print('-------------')
print(df.iloc[sorted_similarity[1]].tweet_text)
print('-------------')
print(df.iloc[sorted_similarity[2]].tweet_text)
print('-------------')
print(df.iloc[sorted_similarity[3]].tweet_text)

.@wesley83 I have a 3G iPhone. After 3 hrs tweeting at #RISE_Austin, it was dead!  I need to upgrade. Plugin stations at #SXSW.
-------------
.@mention I have a 3G iPhone. After 3 hrs tweeting at #RISE_Austin, it was dead!  I need to upgrade. Plugin stations at #SXSW.
-------------
IPhone is dead. Find me on the secret batphone #sxsw.
-------------
The big takeaway from #SXSW interactive - I need an iphone.


Мы получили мощный инструмент для анализа текстов - например, мы случайно нашли дубликат твита

Кроме простого подхода, когда мы вычисляем счётчик вхождения токена, можно вычислять более сложную метрику TF-IDF (term frequency - inverse document frequency), которая вычисляется по следующей формуле для токена $t$ и документа $d$:
$$
\text{tf-idf}(t,d) = \text{tf}(t,d)\cdot\text{idf}(t)
$$

Где $\text{tf}(t,d)$ - элемент матрицы, полученной из `CountVectorizer()`, который мы умножаем на величину $\text{idf}(t)$. 

Эта величина показывает количество документов в корпусе  (наборе текстов), в которых был встречен токен $t$:
$$
\text{idf}(t) = \log\frac{1+N}{1+\text{df(t)}} + 1
$$

где $\text{df}(t)$ - количество документов корпуса, в которых был встречен токен $t$. Таким образом мы понижаем веса у слов, которые встречаются почти во всех документах - такие токены являются неинформативными и мусорными, алгоритм понижает их "важность" для анализа.

Алгоритм TF-IDF лучше подходит для анализа текстов и даёт более высокое качество, но более затратен по вычислениям. Как выбрать между этими алгоритмами?

* если токенов менее 10000 используйте TF-IDF.
* если токенов более 10000 то *попробуйте* использовать TF-IDF, если не получится - возвращайтесь к CountVectorizer.

**Недостатки BoW подхода** Используя алгоритмы вроде Вag of Words, мы теряем порядок слов в тексте, а значит, тексты "i have no cows" и "no, i have cows" будут идентичными после векторизации, хотя и противоположными семантически. Чтобы избежать этой проблемы, можно сделать шаг назад и изменить подход к токенизации: например, использовать N-граммы (комбинации из N последовательных токенов). Обычно по корпусу  (набору текстов) формируются биграммы (последовательности из двух слов) или триграммы (последовательности из трёх слов).

Кроме того, текст можно разбивать не на слова, а на последовательности букв - при таком подходе опечатки будут автоматически учитываться.